In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
arm = moveit_commander.MoveGroupCommander('arm')

[ WARN] [1710557748.301620757]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710557748.306378956]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710557748.306413772]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1710557748.308161977]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1710557748.308209693]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1710557748.469712918]: Kinematics solver doesn't support #attempts any

[ WARN] [1710557751.678301046, 477.486000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 477.529000 according to authority unknown_publisher
[ WARN] [1710557751.678746673, 477.529000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 477.529000 according to authority /pose_integrator
[ WARN] [1710557751.679461988, 477.529000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 477.529000 according to authority /pose_integrator
[ WARN] [1710557751.679546889, 477.529000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 477.529000 according to authority /pose_integrator
[ WARN] [1710557751.679635833, 477.529000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 477.529000 according to authority /pose_integrator
[ WARN] [1710557751.679676176, 477.529000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

[ WARN] [1710557754.765914786, 478.936000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 478.945000 according to authority unknown_publisher
[ WARN] [1710557754.768763376, 478.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 478.945000 according to authority /pose_integrator
[ WARN] [1710557754.768847498, 478.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 478.945000 according to authority /pose_integrator
[ WARN] [1710557754.768902713, 478.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 478.945000 according to authority /pose_integrator
[ WARN] [1710557754.768956312, 478.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 478.945000 according to authority /pose_integrator
[ WARN] [1710557754.769014687, 478.945000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame o

In [ ]:
head.set_joint_values([0.0,-0.65])
arm.set_named_target('go')
arm.go()

In [ ]:

rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
omni_base.move_d_to(target_distance=0.78,target_link='apple')

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
for i in range(len(res.names)):
    if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
        bbM=res.pt_min.data[2*i:2*i+2]
        bbm=res.pt_max.data[2*i:2*i+2]
        bbx=[bbM,bbm]
print(bbx)
plt.imshow(debug_image)

# [(297, 191), (231, 226)]  at pkcp pose #Real Tak
### [(323, 266), (280, 215)]  at pkcp pose #Gazebo


In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eY=(bbx[1][1]-239)/239
    eX=(bbx[0][0]-312)/312
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
bbx

In [ ]:
eY=(bbx[1][1]-239)/239
eX=(bbx[0][0]-312)/312
print(f'error relative to gripper cam ex={eX} ey={eY}')


In [ ]:
#omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 


In [ ]:
clear_octo_client()

av=arm.get_current_joint_values()
print (av,'av')
av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
plt.imshow(hand_cam.get_image())

In [ ]:
image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
plt.imshow(debug_image)

In [ ]:
gripper.open()

In [ ]:
gripper.close(force=0.06)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
    tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])




In [ ]:
def detect_object_yolo(object_name,res):
    # find object_name in the response message from object_classification service (Yolo)
    objs=[]
    for i,name in enumerate(res.names):
        objs.append(name.data[4:])
        if name.data[4:]==object_name:return res.poses[i]
    if object_name=='all':
        
        return objs
    return []

In [ ]:
#img_msg  = bridge.cv2_to_imgmsg(rgbd.get_image())  #REAL
img_msg  = bridge.cv2_to_imgmsg(cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR ))#GAZEB
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   
if len (objects)!=0:
    for i in range(len(res.poses)):
        tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(res.names[i].data[4:])
        rospy.sleep(0.3)
        pose , _=tf_man.getTF(res.names[i].data[4:])
        new_row = {'x': pose[0], 'y': pose[1], 'z': pose[2], 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
objs

In [ ]:
objs[['x','y','z']].values

In [ ]:
#get closest object
rob_pos,_=tf_man.getTF('base_link')
np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))
objs.iloc[np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))]['obj_name']

In [ ]:
objs[objs['obj_name']==objs.obj_name.values[0]].x.values[0]

# PLANE 1
#### 7.883384	5.948245	0.701945,
#### 8.214565	5.962619	0.700034,
#### 8.213697	6.107727	0.709487,
####  7.893623	6.082935	0.693285

# PLANE 2
#### 7.88076	6.086619	0.416096,
#### 8.214128	5.9663	0.416672

# Plane 3
#### 8.20238	5.963262	0.117541

In [ ]:
head.set_named_target('neutral')

av=arm.get_current_joint_values()
av[0]=0.5
av[1]=-0.5
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:


av=arm.get_current_joint_values()
av[0]=0.05

arm.go(av)

In [ ]:
head.set_joint_values([-np.pi/2 , -0.8])

av=arm.get_current_joint_values()
av[0]=0.0
av[1]=-0.5
arm.go(av)

In [4]:
import pandas as pd


In [19]:
import pandas as pd
objs = pd.read_csv ('/home/roboworks/Documents/objs.csv')
objs=objs.drop(columns='Unnamed: 0')

In [20]:
objs

,x,y,z,obj_name,shelves,pickup,category
0,8.540362,-2.302229,0.711838,baseball,False,False,balls
1,8.588709,-2.163747,0.710513,f_cups,False,False,kitchen
2,8.159609,-1.997206,0.006306,spatula,False,True,kitchen
3,6.250804,-0.725341,0.831389,a_marbles,False,False,other
4,7.309552,-3.363026,0.677819,a_marbles,False,False,other
5,8.160490,-2.318883,0.673028,pitcher_lid,False,False,other
6,6.095096,-3.146115,0.258413,master_chef_can,False,False,food
7,8.410027,-2.522980,0.155381,master_chef_can,False,False,food
8,6.402390,-0.752308,0.761527,a_pitcher_base,False,False,other
9,7.283742,-3.308466,0.173525,windex_bottle,False,False,other


In [44]:
from sklearn.cluster import KMeans
shelf_objs=objs[objs['shelves']==True ]
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]


In [82]:
'other' in sh_o_top['category'].values

True

In [91]:
shelf_objs=objs[objs['shelves']==True ]

sh_o_btm=shelf_objs[shelf_objs['z']<0.2]
sh_o_mdl=shelf_objs[(shelf_objs['z']>0.2 )& (shelf_objs['z']<0.5) ]
sh_o_top=shelf_objs[(shelf_objs['z']>0.5 )& (shelf_objs['z']<1.5) ]
sh_o_btm['category'].value_counts(),sh_o_mdl['category'].value_counts() 

a= sh_o_btm['category'].value_counts()
if 'other' in sh_o_btm['category'].values:a.drop('other', inplace=True)
print(f'btm shelf category {a.index[a.argmax()]}')

a= sh_o_mdl['category'].value_counts()
if 'other' in sh_o_mdl['category'].values:a.drop('other', inplace=True)
print(f'middle shelf category {a.index[a.argmax()]}')


a= sh_o_top['category'].value_counts()
if 'other' in sh_o_top['category'].values:a.drop('other', inplace=True)
print(f'top shelf category {a.index[a.argmax()]}')


btm shelf category tools
middle shelf category kitchen
top shelf category kitchen


In [86]:
a

category
tools    1
Name: count, dtype: int64

In [3]:
kitchen =['bowl','spatula','spoon', 'bowl']
tools=['extra_large_clamp','large_clamp','small_clamp','medium_clamp','adjustable_wrench',
'flat_screwdriver','phillips_screwdriver','wood_block']
balls= ['softball','tennis_ball']
fruits= ['apple','banana', 'lemon','pear']
food =['chips_can'
,'mustard_bottle'
,'potted_meat_can'
,'tomato_soup_can'
,'tuna_fish_can']

def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats   


In [4]:
def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
    
    

In [5]:
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats

In [6]:
objs

,x,y,z,obj_name,category
0,5.980992,4.677664,0.506293,a_colored_wood_blocks,other
1,6.219399,4.659809,0.421303,bowl,kitchen
2,5.928290,4.551338,0.290167,b_colored_wood_blocks,other
3,6.390869,4.590297,0.431736,a_pitcher_base,other
4,8.230576,6.101967,0.704379,master_chef_can,other
5,8.221585,5.984989,0.944968,nine_hole_peg_test,other
6,8.032595,5.981796,0.418046,h_cups,other
7,7.947621,6.034316,0.421347,softball,balls
8,8.154552,5.987593,0.688931,tuna_fish_can,food
9,8.119175,6.098844,0.087389,large_clamp,tools


In [7]:
import numpy as np
regions={'shelves':np.load('/home/roboworks/Documents/shelf_sim.npy'),'pickup':np.load('/home/roboworks/Documents/pickup_sim.npy')}
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 
for name in regions:
    in_region=[]
    area_box=regions[name]
    print (area_box,name)
    for index, row in objs[['x','y','z']].iterrows():
        in_region.append(is_inside(row.x, row.y))
        print (row.x, row.y,is_inside(row.x, row.y))
    objs[name]=pd.Series(in_region)
objs

[[8.  6.2]
 [8.6 5.9]] shelves
8.427302439444869 -2.387099938691619 False
4.189883221764755 -3.12817517224511 False
6.996095572401662 -3.385444536385085 False
8.241146517384646 -2.1234714397951437 False
8.385500617275074 -2.6580331149631853 False
6.738603323946117 -3.3399815724440334 False
8.456481766905478 -2.1191276737283378 False
8.412992079952813 -2.376650791173282 False
7.408113251719342 -3.5837193368711446 False
9.827777536725833 -0.1736827784155536 False
8.71789504057108 -0.7650194610032485 False
9.42270083077709 -0.7967892993931813 False
8.72939391900003 -0.8863480440762128 False
9.679823610875626 -0.6422716938996136 False
9.6812992225024 -0.6301467986524352 False
9.599156645696697 -0.5458299868701944 False
9.66885550110473 -0.8080967243181236 False
8.94006119969579 -0.8552727450276483 False
9.427913316426327 -0.8005868059264166 False
8.941879617700575 -0.8571746001411781 False
9.464883890218273 -0.6275242023490577 False
8.941935417049475 -0.8579549063104037 False
9.75603472185

,x,y,z,obj_name,shelves,pickup,category
0,8.427302,-2.387100,0.706535,potted_meat_can,False,False,food
1,4.189883,-3.128175,0.344880,a_toy_airplane,False,False,other
2,6.996096,-3.385445,0.032005,pitcher_lid,False,False,other
3,8.241147,-2.123471,0.722802,tomato_soup_can,False,False,food
4,8.385501,-2.658033,0.125929,master_chef_can,False,False,food
5,6.738603,-3.339982,0.025703,pitcher_lid,False,False,other
6,8.456482,-2.119128,0.693669,racquetball,False,False,balls
7,8.412992,-2.376651,0.697793,potted_meat_can,False,False,food
8,7.408113,-3.583719,0.248668,a_colored_wood_blocks,False,False,other
9,9.827778,-0.173683,1.062830,pudding_box,False,False,food


In [42]:
(area_box[:,1].max() > row.y),(area_box[:,1].min() < row.y),((area_box[:,0].max() > row.x) , (area_box[0,0].min() < row.x)) 

(True, True, (True, True))

In [23]:
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 

,x,y,z


In [9]:
pickup_region=np.zeros((2,2))
#np.save('/home/roboworks/Documents/pickup_region.npy', pickup_region)

In [30]:
ab[0,0],ab[0,1]=6,6
ab[1,0],ab[1,1]=7,4.0

In [1]:
import numpy as np

#ab=np.load('/home/roboworks/Documents/pickup_region.npy')
ab= np.load('/home/roboworks/Documents/pickup_sim.npy')

In [4]:

ab[0,0]=5.8
ab

array([[5.8, 6. ],
       [7. , 4. ]])

In [35]:
np.save('/home/roboworks/Documents/pickup_region.npy',ab)


In [5]:
#np.save('/home/roboworks/Documents/shelf_sim.npy',ab)
np.save('/home/roboworks/Documents/pickup_sim.npy',ab)

In [93]:
objs[objs['pickup']==True][['x','y','z']].values

array([[6.17398135, 4.65753106, 0.42089542],
       [6.36147013, 4.59256394, 0.44251327],
       [6.36858971, 4.54830806, 0.29265981],
       [6.46370772, 4.5543959 , 0.28693223],
       [6.47802778, 4.65070459, 0.40397941],
       [6.29717109, 4.55135968, 0.28950038],
       [6.34149378, 4.58114838, 0.42872859],
       [6.1789658 , 4.6589119 , 0.42126921]])

In [106]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
ix


[ WARN] [1710473773.101862197]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101925102]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101952618]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.101978752]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.102010881]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator
[ WARN] [1710473773.102037519]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.224000 according to authority /pose_integrator


4

[ WARN] [1710473773.290014351]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290070224]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290111760]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290147588]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290181250]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.290212982]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2958.307000 according to authority /pose_integrator
[ WARN] [1710473773.74

[ WARN] [1710473776.715985886]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716047074]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716083172]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716109801]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716134253]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473776.716159524]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2959.789000 according to authority /pose_integrator
[ WARN] [1710473778.05

obj_name    spatula
category    kitchen
Name: 6, dtype: object

[ WARN] [1710473806.863826319]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863869473]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863894426]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863932673]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863964885]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473806.863994091]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2972.635000 according to authority /pose_integrator
[ WARN] [1710473807.09

[ WARN] [1710473810.481276302]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481338679]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481390508]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481440216]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481483740]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473810.481528618]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2974.274000 according to authority /pose_integrator
[ WARN] [1710473811.05

[ WARN] [1710473814.770515112]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770570099]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770602751]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770648316]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770683081]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473814.770721679]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2976.366000 according to authority /pose_integrator
[ WARN] [1710473815.20

[ WARN] [1710473818.818562271]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818614164]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818654467]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818690759]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818728534]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473818.818763216]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2978.258000 according to authority /pose_integrator
[ WARN] [1710473819.23

...

[ WARN] [1710473890.908382378]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908437390]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908473499]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908521639]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908559844]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473890.908597533]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2983.908000 according to authority /pose_integrator
[ WARN] [1710473892.06

[ WARN] [1710473897.962512149]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962573494]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962618521]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962659185]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962703576]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473897.962755295]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2986.775000 according to authority /pose_integrator
[ WARN] [1710473898.80

[ WARN] [1710473902.428122221]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428172153]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428197732]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428222960]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428248417]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.428272107]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2988.817000 according to authority /pose_integrator
[ WARN] [1710473902.54

[ WARN] [1710473905.507204782]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507270158]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507307681]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507346708]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507377484]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.507410480]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2990.186000 according to authority /pose_integrator
[ WARN] [1710473905.77

[ WARN] [1710473908.294075612]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294122461]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294151826]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294178525]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294206591]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.294233846]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2991.412000 according to authority /pose_integrator
[ WARN] [1710473908.55

In [111]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
objs[objs['pickup']==True][['obj_name','category']].iloc[ix]


obj_name    spatula
category    kitchen
Name: 6, dtype: object

In [112]:
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))

In [13]:
pose=np.asarray((8.212561, 6.028789, 0.12))
pose

array([8.212561, 6.028789, 0.12    ])

In [14]:
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

[ WARN] [1710721819.982457468]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982510844]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982544395]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982569973]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982593795]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721819.982616974]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.832000 according to authority /pose_integrator
[ WARN] [1710721833.81

[ WARN] [1710721890.117906049]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.117971106]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118015771]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118073132]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118106900]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721890.118132299]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2301.721000 according to authority /pose_integrator
[ WARN] [1710721895.90

In [2]:
#pose=np.asarray((8.212561, 6.028789, 0.12))
pose=np.asarray((8.062455, 6.0622816, 0.6610926))
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

[ WARN] [1710781382.841518748]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841577977]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841610689]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841640915]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841670497]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator
[ WARN] [1710781382.841699811]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 717.383000 according to authority /pose_integrator


shutdown request: [/smach] Reason: new node registered with same name
